In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM,DIN
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
import numpy as np
import tensorflow as tf
import random
import pickle
import keras
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
def load_variable(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r
gid_dict = load_variable('/home/jovyan/dataset/gid_dict')
gid_list = load_variable('/home/jovyan/dataset/gid_list')#len = 1739595
df_val = pd.read_csv('/home/jovyan/dataset/validate_folder/502val.csv')
df_candidate = load_variable('/home/jovyan/dataset/original_candidate_itemset/0502_candidate_itemset')#len = 1422941

In [2]:
def clean_candidate(df):#舊方法刪完後的結果(不存在就刪掉)
    df = df[df['gid'].isin(gid_list)]
    df.candidate_items = df.candidate_items.apply(lambda x:x.split(','))
    df = df.explode('candidate_items', ignore_index=True)
    df = df[df['candidate_items'].isin(gid_list)]#推薦清單 candidate_item清掉不在4/1-4/30的商品
    df.gid = df.gid.apply(lambda x:gid_dict[x])
    df.candidate_items = df.candidate_items.apply(lambda x:gid_dict[x])
    df = df.groupby(['gid']).agg({'candidate_items': lambda x: x.tolist()})
    df = df[df.candidate_items.apply(lambda x:len(x)) >15]
    df = df.reset_index()
    return df
df_candidate = clean_candidate(df_candidate)#888586
df_candidate['original_score'] = df_candidate.candidate_items.apply(lambda x:list(np.arange(1,0,-1/len(x))))
print(len(df_candidate))

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


888586


In [3]:
df5_all = load_variable('/home/jovyan/dataset/df5_all')
df_uid = df5_all[df5_all['uid'] != 'unknow']
df_session = df5_all[df5_all['uid'] == 'unknow']
lbe = LabelEncoder()
for i in ['uid']:
    df_uid['uid_encode'] = lbe.fit_transform(df_uid[i])
    df_uid['uid_encode'] = df_uid['uid_encode'].apply(lambda x:x+1)
for i in ['ven_session']:
    df_session['session_encode'] = lbe.fit_transform(df_session[i])
    df_session['session_encode'] = df_session['session_encode'].apply(lambda x:x+1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [5]:
uid_encode_unique = df_uid[['uid','uid_encode']].sort_values('uid_encode').uid_encode.unique()
uid_unique = df_uid[['uid','uid_encode']].sort_values('uid_encode').uid.unique()
uid_dict = {}
for i in zip(uid_encode_unique,uid_unique):
    uid_dict[i[1]] = i[0]
session_encode_unique = df_session[['ven_session','session_encode']].sort_values('session_encode').session_encode.unique()
session_unique = df_session[['ven_session','session_encode']].sort_values('session_encode').ven_session.unique()
session_dict = {}
for i in zip(session_encode_unique,session_unique):
    session_dict[i[1]] = i[0]
session_dict[0] = 0
uid_dict[0] = 0

In [6]:
df_uid5 = df_uid[df_uid['month'] == 52]
df_session5 = df_session[df_session['month'] == 52]
df_uid5.uid = df_uid5.uid.apply(lambda x:uid_dict[x])
df_session5.ven_session = df_session5.ven_session.apply(lambda x:session_dict[x])
df_session5 = df_session5.drop(columns = ['uid','ven_guid'])
df_uid5 = df_uid5.drop(columns=['ven_guid','ven_session'])

In [7]:
df_val_session = df_val[df_val.uid.isna()]
df_val_uid = df_val[~df_val.uid.isna()]
len(df_val_session),len(df_val_uid)

(88877, 44750)

In [8]:
df_val_session.ven_session = df_val_session.ven_session.apply(lambda x:session_dict[x])
df_val_uid.uid = df_val_uid.uid.apply(lambda x:uid_dict[x])
df_val_session.gid = df_val_session.gid.apply(lambda x:gid_dict[x])
df_val_uid.gid = df_val_uid.gid.apply(lambda x:gid_dict[x])

In [9]:
def process(df_val,df,idname):
    hist = []
    id_list = df_val[idname].unique().tolist()
    for i in id_list:
        tempdf = df[df[idname] == i]
        tempdf_copy = df[df[idname] == i]
        gid_list = df_val[df_val[idname] == i].gid.tolist()
        for j in gid_list:
            index = tempdf[tempdf.gid_encode == j].index[0]
            tempdf = tempdf.drop(index)
            hist.append(tempdf_copy.loc[:index].gid_encode.tolist())
    df_val['hist'] = hist
    return df_val
df_val_session = process(df_val_session,df_session5,'ven_session')
df_val_uid = process(df_val_uid,df_uid5,'uid')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [48]:
result_session = pd.merge(df_candidate,df_val_session,how = 'inner',on = ['gid'])
result_uid = pd.merge(df_candidate,df_val_uid,how = 'inner',on = ['gid'])
result_session = result_session[['ven_session','gid','hist','next_gid','candidate_items','original_score']]
result_uid = result_uid[['uid','gid','hist','next_gid','candidate_items','original_score']]
result_session = result_session.sort_values('ven_session')
result_uid = result_uid.sort_values('uid')
result_session.next_gid = result_session.next_gid.apply(lambda x:gid_dict[x])
result_uid.next_gid = result_uid.next_gid.apply(lambda x:gid_dict[x])
result_session['flag'] = list(range(1,len(result_session)+1))
result_uid['flag'] = list(range(1,len(result_uid)+1))
print(len(result_session),len(result_uid)) #(62166, 32886)
result_session_explode = result_session.explode(['candidate_items','original_score'], ignore_index=True)
result_uid_explode = result_uid.explode(['candidate_items','original_score'], ignore_index=True)

58396 30518


In [36]:
def create_pred_set(result):
    pred_list = []
    hist = 0
    for i in zip(result['hist'].tolist(),result['candidate_items'].tolist()):
        if len(i[0])>100:
            pred_list.append((i[0][-100:],i[1],100))
        else:
            pred_list.append((i[0],i[1],len(i[0])))
    return pred_list
pred_set_session = create_pred_set(result_session_explode)
pred_set_uid = create_pred_set(result_uid_explode)

In [37]:
def create_predict_array(pred_set):
    hist_g_id = []
    g_id = []
    seq_length = []
    for index, value in enumerate(pred_set):
        hist_g_id.append(value[0]) 
        g_id.append(value[1])
        seq_length.append(value[2])
    for i in range(len(hist_g_id)):
        if len(hist_g_id[i]) < 100:
            hist_g_id[i] += (100-len(hist_g_id[i]))*[0]
    g_id = np.array(g_id)
    hist_g_id = np.array(hist_g_id)
    seq_length = np.array(seq_length)
    return hist_g_id, g_id, seq_length

hist_gid_session, gid_session, seq_length_session = create_predict_array(pred_set_session)
hist_gid_uid, gid_uid, seq_length_uid = create_predict_array(pred_set_uid)

In [38]:
x_uid_pred = {'g_id': gid_uid, 
        'hist_g_id': hist_gid_uid, 
        'seq_length': seq_length_uid}
x_session_pred = {'g_id': gid_session, 
        'hist_g_id': hist_gid_session, 
        'seq_length': seq_length_session}

### 4/1-5/1 predict 5/2

In [39]:
behavior_feature_list = ["g_id"]
feature_columns = [SparseFeat('g_id', 1870599, embedding_dim=8)]
feature_columns += [VarLenSparseFeat(SparseFeat('hist_g_id',vocabulary_size=1870599, embedding_dim=8, embedding_name='g_id'), maxlen = 100, length_name='seq_length')]

In [40]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/checkpoint_folder/model_401_501")
model.load_weights(latest)

In [41]:
y_uid = model.predict(x_uid_pred)
y_session = model.predict(x_session_pred)
y_uid = y_uid.reshape(-1).tolist()
y_session = y_session.reshape(-1).tolist()
result_uid_explode['score'] = y_uid
result_session_explode['score'] = y_session

In [59]:
def cal_new_score(result):
    new_score = []
    for i in result.iterrows():
        new_score.append(0.2*(i[1]['score'])+0.8*(i[1]['original_score']))
    result['new_score'] = new_score
    result = result.sort_values(['flag','new_score'],ascending = False)
    df_output = result.groupby(['flag','gid','next_gid'],as_index=False).agg({'candidate_items': lambda x: x.tolist()})
    return df_output
df1 = cal_new_score(result_uid_explode)
df2 = cal_new_score(result_session_explode)
resultdf = pd.concat([df1,df2],ignore_index=True)

In [60]:
top5 = hitrate(resultdf,5)
top10 = hitrate(resultdf,10)
top15 = hitrate(resultdf,15)
print(top5,top10,top15)

0.23767910565265313 0.33142137346199696 0.38098612142069865


In [52]:
def hitrate(result,k):
    count = 0
    for i in result.iterrows():
        if i[1]['next_gid'] in i[1]['candidate_items'][:k]:
            count+=1
    return count/len(result)
top5 = hitrate(resultdf,5)
top10 = hitrate(resultdf,10)
top15 = hitrate(resultdf,15)
print(top5,top10,top15)

0.2308522842297051 0.3285421868322199 0.37994016690284993


#### using epoch = 15 model predict

In [68]:
model15 = DIN(feature_columns, behavior_feature_list)
model15.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/15epoch/model401_501")
model15.load_weights(latest)

In [72]:
y_uid15 = model15.predict(x_uid_pred)
y_session15 = model15.predict(x_session_pred)
y_uid15 = y_uid15.reshape(-1).tolist()
y_session15 = y_session15.reshape(-1).tolist()
result_uid_explode['score'] = y_uid15
result_session_explode['score'] = y_session15

AttributeError: 'list' object has no attribute 'reshape'

In [79]:
epoch15_df1 = result_uid_explode.sort_values(['flag','score'],ascending = False)
epoch15_df2 = result_session_explode.sort_values(['flag','score'],ascending = False)
epoch15_df1 = epoch15_df1.groupby(['flag','gid','next_gid'],as_index=False).agg({'candidate_items': lambda x: x.tolist()})
epoch15_df2 = epoch15_df2.groupby(['flag','gid','next_gid'],as_index=False).agg({'candidate_items': lambda x: x.tolist()})
epoch15_df = pd.concat([epoch15_df1,epoch15_df2],ignore_index=True)

In [88]:
top5 = hitrate(epoch15_df,5)
top10 = hitrate(epoch15_df,10)
top15 = hitrate(epoch15_df,15)
print(top5,top10,top15)

0.14648986661268193 0.2387700474615921 0.3119981105337742


#### using 20 epoch model to predict

In [89]:
model20 = DIN(feature_columns, behavior_feature_list)
model20.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/20epoch/model401_501")
model20.load_weights(latest)

y_uid20 = model20.predict(x_uid_pred)
y_session20 = model20.predict(x_session_pred)
y_uid20 = y_uid20.reshape(-1).tolist()
y_session20 = y_session20.reshape(-1).tolist()
result_uid_explode['score'] = y_uid20
result_session_explode['score'] = y_session20

In [92]:
epoch20_df1 = result_uid_explode.sort_values(['flag','score'],ascending = False)
epoch20_df2 = result_session_explode.sort_values(['flag','score'],ascending = False)
epoch20_df1 = epoch20_df1.groupby(['flag','gid','next_gid'],as_index=False).agg({'candidate_items': lambda x: x.tolist()})
epoch20_df2 = epoch20_df2.groupby(['flag','gid','next_gid'],as_index=False).agg({'candidate_items': lambda x: x.tolist()})
epoch20_df = pd.concat([epoch20_df1,epoch20_df2],ignore_index=True)

top5 = hitrate(epoch20_df,5)
top10 = hitrate(epoch20_df,10)
top15 = hitrate(epoch20_df,15)
print(top5,top10,top15)

0.14536518433542525 0.2352835324020964 0.30944508176440155


### 4/1-5/2 model predict 5/2

In [54]:
model502 = keras.models.load_model("/home/jovyan/dataset/rakuten_model_401_502")
y_uid52 = model502.predict(x_uid_pred)
y_session52 = model502.predict(x_session_pred)
y_uid52 = y_uid52.reshape(-1).tolist()
y_session52 = y_session52.reshape(-1).tolist()

In [62]:
result_uid_explode['score'] = y_uid52
result_session_explode['score'] = y_session52

In [63]:
def cal_new_score(result):
    new_score = []
    for i in result.iterrows():
        new_score.append(0.2*(i[1]['score'])+0.8*(i[1]['original_score']))
    result['new_score'] = new_score
    result = result.sort_values(['flag','new_score'],ascending = False)
    df_output = result.groupby(['flag','gid','next_gid'],as_index=False).agg({'candidate_items': lambda x: x.tolist()})
    return df_output
df3 = cal_new_score(result_uid_explode)
df4 = cal_new_score(result_session_explode)
resultdf = pd.concat([df3,df4],ignore_index=True)

In [64]:
top5 = hitrate(resultdf,5)
top10 = hitrate(resultdf,10)
top15 = hitrate(resultdf,15)
print(top5,top10,top15)

0.26090379467800345 0.34344422700587085 0.3884090244505927


In [57]:
def hitrate(result,k):
    count = 0
    for i in result.iterrows():
        if i[1]['next_gid'] in i[1]['candidate_items'][:k]:
            count+=1
    return count/len(result)
top5 = hitrate(resultdf,5)
top10 = hitrate(resultdf,10)
top15 = hitrate(resultdf,15)
print(top5,top10,top15)

0.2680680207841285 0.35038351665654455 0.39309894954675306


In [24]:
top5_session2 = hitrate(result_session_explode,5,'ven_session')
top5_uid2 = hitrate(result_uid_explode,5,'uid')
top10_session2 = hitrate(result_session_explode,10,'ven_session')
top10_uid2 = hitrate(result_uid_explode,10,'uid')
top15_session2 = hitrate(result_session_explode,15,'ven_session')
top15_uid2 = hitrate(result_uid_explode,15,'uid')
print((top5_session2+top5_uid2)/total_len)
print((top10_session2+top10_uid2)/total_len)
print((top15_session2+top15_uid2)/total_len)

0.20029466675664126
0.298738106484918
0.35629934543491465


#### 15 epoch predict

In [93]:
model502 = DIN(feature_columns, behavior_feature_list)
model502.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/15epoch/model401_502")
model502.load_weights(latest)

y_uid15 = model502.predict(x_uid_pred)
y_session15 = model502.predict(x_session_pred)
y_uid15 = y_uid15.reshape(-1).tolist()
y_session15 = y_session15.reshape(-1).tolist()
result_uid_explode['score'] = y_uid15
result_session_explode['score'] = y_session15

In [94]:
epoch15_df1 = result_uid_explode.sort_values(['flag','score'],ascending = False)
epoch15_df2 = result_session_explode.sort_values(['flag','score'],ascending = False)
epoch15_df1 = epoch15_df1.groupby(['flag','gid','next_gid'],as_index=False).agg({'candidate_items': lambda x: x.tolist()})
epoch15_df2 = epoch15_df2.groupby(['flag','gid','next_gid'],as_index=False).agg({'candidate_items': lambda x: x.tolist()})
epoch15_df = pd.concat([epoch15_df1,epoch15_df2],ignore_index=True)
top5 = hitrate(epoch15_df,5)
top10 = hitrate(epoch15_df,10)
top15 = hitrate(epoch15_df,15)
print(top5,top10,top15)

0.2563825719234316 0.346165958116832 0.3901747756258857


#### 20 epoch predict

In [95]:
model502 = DIN(feature_columns, behavior_feature_list)
model502.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/20epoch/model401_502")
model502.load_weights(latest)

y_uid20 = model502.predict(x_uid_pred)
y_session20 = model502.predict(x_session_pred)
y_uid20 = y_uid20.reshape(-1).tolist()
y_session20 = y_session20.reshape(-1).tolist()
result_uid_explode['score'] = y_uid20
result_session_explode['score'] = y_session20

In [96]:
epoch20_df1 = result_uid_explode.sort_values(['flag','score'],ascending = False)
epoch20_df2 = result_session_explode.sort_values(['flag','score'],ascending = False)
epoch20_df1 = epoch20_df1.groupby(['flag','gid','next_gid'],as_index=False).agg({'candidate_items': lambda x: x.tolist()})
epoch20_df2 = epoch20_df2.groupby(['flag','gid','next_gid'],as_index=False).agg({'candidate_items': lambda x: x.tolist()})
epoch20_df = pd.concat([epoch20_df1,epoch20_df2],ignore_index=True)

top5 = hitrate(epoch20_df,5)
top10 = hitrate(epoch20_df,10)
top15 = hitrate(epoch20_df,15)
print(top5,top10,top15)

0.25295229097779875 0.34387160627122837 0.38792541107137235


### baseline

In [11]:
df_val1 = df_val_session[df_val_session.gid.isin(df_candidate.gid.tolist())]
df_val2 = df_val_uid[df_val_uid.gid.isin(df_candidate.gid.tolist())]
df_val_baseline =  pd.concat([df_val1,df_val2],ignore_index=True)
df_val_baseline.next_gid = df_val_baseline.next_gid.apply(lambda x:gid_dict[x])
df_val_baseline = pd.merge(df_candidate,df_val_baseline,how = 'inner',on = ['gid'])

In [16]:
def hitrate_baseline(df,k):
    count = 0
    for i in df.iterrows():
        if i[1]['next_gid'] in i[1]['candidate_items'][:k]:
            count += 1 
    return count/len(df)

In [17]:
top5_b = hitrate_baseline(df_val_baseline,5)
top10_b = hitrate_baseline(df_val_baseline,10) 
top15_b = hitrate_baseline(df_val_baseline,15) 
print(top5_b,top10_b,top15_b)

0.23869131970218413 0.32898081292035 0.379782711384034
